<a href="https://colab.research.google.com/github/OmkarPatkar/Text-Classification/blob/master/Text_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# ** Build a Basic ML Model for Text Classification **

* Here we will build a machine learning model for text classification.

* We will build a machine learning model to classify whether a particular tweet is hate speech or not.


In [1]:
#import the library
#Load the dataset
import pandas as pd

dataset = pd.read_csv('drive/My Drive/Colab Notebooks/Basic ML Model for Text Classification/final_dataset_basicmlmodel.csv')
dataset.head()

,id,label,tweet
0,1,0,@user when a father is dysfunctional and is s...
1,2,0,@user @user thanks for #lyft credit i can't us...
2,3,0,bihday your majesty
3,4,0,#model i love u take with u all the time in ...
4,5,0,factsguide: society now #motivation



* label is the column that contains the target variable or the value that has to be predicted. 1 means it's a hate speech and 0 means it is not.
    
* tweet is the column that contains the text of the tweet. This is the main data on which NLP techniques will be applied.


In [2]:
#Let's check out some of the tweets.
for index, tweet in enumerate(dataset["tweet"][10:15]):
  print(index+1, ".", tweet)

1 .  â #ireland consumer price index (mom) climbed from previous 0.2% to 0.5% in may   #blog #silver #gold #forex
2 . we are so selfish. #orlando #standwithorlando #pulseshooting #orlandoshooting #biggerproblems #selfish #heabreaking   #values #love #
3 . i get to see my daddy today!!   #80days #gettingfed
4 . ouch...junior is angryð#got7 #junior #yugyoem   #omg 
5 . i am thankful for having a paner. #thankful #positive     


In [0]:
#Let's clean up the noise in our dataset.
import re

#Clean text from noise
def clean_text(text):
  #Filter to allow only alphabets
  text = re.sub(r'[^a-zA-Z\']', ' ', text )

  #Remove Unicode characters
  text = re.sub(r'[^\x00-\x7F]+', '', text)
    
  #Convert to lowercase to maintain consistency
  text = text.lower()
       
  return text

In [0]:
dataset['clean_text'] = dataset.tweet.apply(lambda x : clean_text(x))

we are going to do Feature Engieering.

Feature engineering is the science (and art) of extracting more information from existing data.

The machine learning model does not understand text directly, so we create numerical features that reperesant the underlying text.

In [0]:
#Exhaustive list of stopwords in the english language.
#We want to focus less on these so at some point will have to filter

STOP_WORDS = ['a', 'about', 'above', 'after', 'again', 'against', 'all', 'also', 'am', 'an', 'and',
              'any', 'are', "aren't", 'as', 'at', 'be', 'because', 'been', 'before', 'being', 'below',
              'between', 'both', 'but', 'by', 'can', "can't", 'cannot', 'com', 'could', "couldn't", 'did',
              "didn't", 'do', 'does', "doesn't", 'doing', "don't", 'down', 'during', 'each', 'else', 'ever',
              'few', 'for', 'from', 'further', 'get', 'had', "hadn't", 'has', "hasn't", 'have', "haven't", 'having',
              'he', "he'd", "he'll", "he's", 'her', 'here', "here's", 'hers', 'herself', 'him', 'himself', 'his', 'how',
              "how's", 'however', 'http', 'i', "i'd", "i'll", "i'm", "i've", 'if', 'in', 'into', 'is', "isn't", 'it',
              "it's", 'its', 'itself', 'just', 'k', "let's", 'like', 'me', 'more', 'most', "mustn't", 'my', 'myself',
              'no', 'nor', 'not', 'of', 'off', 'on', 'once', 'only', 'or', 'other', 'otherwise', 'ought', 'our', 'ours',
              'ourselves', 'out', 'over', 'own', 'r', 'same', 'shall', "shan't", 'she', "she'd", "she'll", "she's",
              'should', "shouldn't", 'since', 'so', 'some', 'such', 'than', 'that', "that's", 'the', 'their', 'theirs',
              'them', 'themselves', 'then', 'there', "there's", 'these', 'they', "they'd", "they'll", "they're",
              "they've", 'this', 'those', 'through', 'to', 'too', 'under', 'until', 'up', 'very', 'was', "wasn't",
              'we', "we'd", "we'll", "we're", "we've", 'were', "weren't", 'what', "what's", 'when', "when's", 'where',
              "where's", 'which', 'while', 'who', "who's", 'whom', 'why', "why's", 'with', "won't", 'would', "wouldn't",
              'www', 'you', "you'd", "you'll", "you're", "you've", 'your', 'yours', 'yourself', 'yourselves']

#Generate word frequency
def gen_freq(text):
  #Will store the list of words
  word_list = []

  #Loop over all the tweets and extract words into word_list
  for tw_word in text.split():
    word_list.extend(tw_word)

  #Create word frequencies using word_list
  word_freq = pd.Series(word_list).value_counts()

  #Drop the stopwords during the frequency calculation
  word_freq = word_freq.drop(STOP_WORDS, errors = 'ignore')

  return word_freq

#Check whether a negation term is present in the text
def any_neg(words):
  for word in words:
    if word in['n', 'no', 'non', 'not'] or re.search(r"\wn't", word):
      return 1
    else:
      return 0

#Check whether one of the 100 rare words is present in the text
def any_rare(words, rare_100):
  for word in words:
    if word in rare_100:
      return 1
    else:
      return 0

#Check whether prompt words are present
def is_question(words):
  for word in words:
    if word in ['when', 'what', 'how', 'why', 'who']:
      return 1
    else:
      return 0

In [0]:
word_freq = gen_freq(dataset.clean_text.str)

#100 most rare words in the dataset
rare_100 = word_freq[-100:]

#Number of words in a tweet
dataset['word_count'] = dataset.clean_text.str.split().apply(lambda x: len(x))

#Negation present or not
dataset['any_neg'] = dataset.clean_text.str.split().apply(lambda x: any_neg(x))

#Prompt present or not
dataset['is_question'] = dataset.clean_text.str.split().apply(lambda x: is_question(x))

#Any of the most 100 rare words present or not
dataset['any_rare'] = dataset.clean_text.str.split().apply(lambda x: any_rare(x, rare_100))

#Character count of the tweet
dataset['char_count'] = dataset.clean_text.apply(lambda x: len(x))

In [7]:
#Top 10 common words are
gen_freq(dataset.clean_text.str)[:10]

user      3351
amp        439
love       320
day        254
trump      214
happy      207
will       191
people     186
new        171
u          158
dtype: int64

In [8]:
dataset.head()

,id,label,tweet,clean_text,word_count,any_neg,is_question,any_rare,char_count
0,1,0,@user when a father is dysfunctional and is s...,user when a father is dysfunctional and is s...,18,0,0,0,102
1,2,0,@user @user thanks for #lyft credit i can't us...,user user thanks for lyft credit i can't us...,19,0,0,0,122
2,3,0,bihday your majesty,bihday your majesty,3,0,0,0,21
3,4,0,#model i love u take with u all the time in ...,model i love u take with u all the time in ...,12,0,0,0,86
4,5,0,factsguide: society now #motivation,factsguide society now motivation,4,0,0,0,39


**Splitting the dataset into Train-Test split**

Let's split the dataset into train and test sets.

Train set is used to train the model and test set is used to make predictions.

In [0]:
from sklearn.model_selection import train_test_split

x = dataset[['word_count', 'any_neg', 'any_rare', 'char_count', 'is_question']]
y = dataset.label

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.1, random_state = 70)

**Train an ML model for Text Classification**

So the dataset is ready, we can use it to train the machine learning model.

We will be using a Naive Bayes classifier from sklearn which is a prominent python library used for machine learning.

In [0]:
from sklearn.naive_bayes import GaussianNB

#Initialize GaussianNB classifier
model = GaussianNB()

#Fit the model on the train dataset
model = model.fit(x_train, y_train)

#Make predictions on the test dataset
pred = model.predict(x_test)

**Evaluate the ML model**

In [23]:
from sklearn.metrics import accuracy_score

print("Accuracy : ", accuracy_score(y_test,pred)*100, "%")

Accuracy :  43.80952380952381 %
